In [11]:
from os.path import join
import os
import pandas as pd

In [12]:
DIR = "/data/NewsGuard/bi-monthly/"

In [13]:
csv_files = [f for f in os.listdir(DIR) \
                if f.endswith(".csv")] 

df_total = pd.DataFrame()
for csv in csv_files: #taking all files in the folder
    df = pd.read_csv(join(DIR, csv))
    df.name=csv
    #print(df.shape)
    # taking name of file, removing the extension and "metadata-"
    df["file_date"]=csv.split(".")[0].split("-")[1]
    # converting file_date to datetime, with format YYYYMMDD00
    df["file_date"]=pd.to_datetime(df["file_date"], 
                                   format="%Y%m%d%H")
    df_total = pd.concat([df_total, df], axis=0)

In [14]:
csv_files = [f for f in os.listdir(DIR) \
                if f.endswith("1501.csv")] 
df_15 = pd.DataFrame()
for csv in csv_files:
    df = pd.read_csv(join(DIR, csv))
    df.name=csv
    #print(df.shape)
    df["file_date"]=csv.split(".")[0].split("-")[1]
    df["file_date"]=pd.to_datetime(df["file_date"], 
                                   format="%Y%m%d%H")
    df_15 = pd.concat([df_15, df], axis=0)

In [15]:
csv_files = [f for f in os.listdir(DIR) \
                if f.endswith("0201.csv")] 
df_02 = pd.DataFrame()
for csv in csv_files:
    df = pd.read_csv(join(DIR, csv))
    df.name=csv
    #print(df.shape)
    df["file_date"]=csv.split(".")[0].split("-")[1]
    df["file_date"]=pd.to_datetime(df["file_date"], 
                                   format="%Y%m%d%H")
    df_02 = pd.concat([df_02, df], axis=0)

In [16]:
df_first = pd.read_csv("/data/NewsGuard/2019/metadata-2019022701.csv",
                        low_memory=False)

In [17]:
#dynamically read in the last file in the repository
df_recent = pd.read_csv("/data/NewsGuard/2024/metadata-2024091601.csv",
                    low_memory=False)

In [18]:
print(f'In the most recent file, there are {len(df_recent)} entries, '
      f'with {df_recent["Domain"].nunique()} unique domains.')

print(f'In the first file, there are {len(df_first)} entries, '
      f'with {df_first["Domain"].nunique()} unique domains.')

In the most recent file, there are 12288 entries, with 10862 unique domains.
In the first file, there are 2375 entries, with 2371 unique domains.


In [19]:
print(f'Number of rows with Score: {len(df_recent) - df_recent.Score.isna().sum()}')
print(f'Number of rows without Score: {df_recent.Score.isna().sum()}')
print(f'Percentage: {round(df_recent.Score.isna().sum()/len(df_recent)*100,2)}%')

Number of rows with Score: 11349
Number of rows without Score: 939
Percentage: 7.64%


In [20]:
rating_counts = df_recent.Rating\
    .value_counts(normalize=False)\
    .reset_index()\
    .rename(columns={"count":"Count"})

rating_countsp = df_recent.Rating\
    .value_counts(normalize=True)\
    .reset_index()\
    .rename(columns={"proportion":"Percent"})

ratings = pd.merge(rating_counts, rating_countsp, 
                   on="Rating")

rating_mean = df_recent\
    .groupby("Rating").Score\
    .mean()\
    .reset_index()\
    .rename(columns={"Score":"Mean"})

pd.merge(ratings, rating_mean, on="Rating")

,Rating,Count,Percent,Mean
0,T,6827,0.555583,87.224769
1,N,4522,0.368001,26.103273
2,FL,806,0.065592,NaN
3,P,70,0.005697,NaN
4,S,63,0.005127,NaN


In [21]:
#for rating in df_recent.Rating.unique():
#    print(rating, df_recent[df_recent.Rating == rating].Domain.unique()[:3])

In [22]:
p_rating = df_recent[df_recent["Rating"] == "FL"]
if not p_rating["Score"].isnull().all():
    mean_score = p_rating["Score"].mean(skipna=True)
    print(f"Mean score for rating FL: {mean_score}")
else:
    print(f"No scores available for rating FL.")

No scores available for rating FL.


In [23]:
#save
df_total.to_csv("/data/NewsGuard/newsguard_twice_month.csv", index=False)
df_15.to_csv("/data/NewsGuard/newsguard_mid_month.csv", index=False)
df_02.to_csv("/data/NewsGuard/newsguard_first_month.csv", index=False)
df_first.to_csv("/data/NewsGuard/newsguard_first_version.csv", index=False)
df_recent.to_csv("/data/NewsGuard/newsguard_recent_version.csv", index=False)